In [1]:
from itertools import chain
import os
import random

from datasets import load_dataset, DatasetDict, Dataset, load_from_disk, concatenate_datasets
import pandas as pd
import preprocess
from tqdm import tqdm
from transformers import AutoTokenizer

/home/knpu/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#context_length = 4098
context_length = 2048

In [3]:
tokenizer = AutoTokenizer.from_pretrained('./tokenizer/tokenizer_v1/')
tokenizer

GPT2TokenizerFast(name_or_path='./tokenizer/tokenizer_v1/', vocab_size=51200, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("<usr>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("<sys>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: 

In [4]:
tokenizer.eos_token_id

1

In [19]:
def tokenize1(batch):
    outputs = tokenizer(
        batch['document'],
        max_length=context_length,
        truncation=True,
        return_overflowing_tokens=True,
        return_length=True,
    )
    
    input_batch = []
    attention_batch = []
    for length, input_ids, attention_masks in zip(outputs['length'], outputs['input_ids'], outputs['attention_mask']):
        if length==context_length:
            input_batch.append(input_ids)
            attention_batch.append(attention_masks)
    return {"input_ids": input_batch, "attention_mask": attention_batch}

In [57]:
def tokenize2(batch):
    outputs = tokenizer(
        batch['document']
    )

    new_input_ids = []
    for input_ids in outputs['input_ids']:
        input_ids.append(tokenizer.encode('\n'))
        new_input_ids.extend(input_ids)
    new_input_ids = new_input_ids[:-1]

    new_input_ids2 = []
    new_attention_mask2 = []
    for i in range(0, len(new_input_ids), context_length):
        if (i + context_length) <= len(new_input_ids):
            new_input_ids2.append(new_input_ids[i:i+context_length])
            new_attention_mask2.append([1] * context_length)

    return {"input_ids": new_input_ids2, "attention_mask": new_attention_mask2}

In [17]:
new_line = tokenizer.encode('\n')
def tokenize3(batch):
    outputs = tokenizer(
        batch['document']
    )

    new_input_ids = []
    for input_ids in outputs['input_ids']:
        input_ids.extend(new_line)
        new_input_ids.extend(input_ids)
    new_input_ids = new_input_ids[:-1]

    new_input_ids2 = []
    new_attention_mask2 = []

    total_length = len(new_input_ids)
    total_length = (total_length // context_length) * context_length
    for i in range(0, total_length, context_length):
        new_input_ids2.append(new_input_ids[i:i+context_length])
        new_attention_mask2.append([1] * context_length)

    return {"input_ids": new_input_ids2, "attention_mask": new_attention_mask2}

In [6]:
dataset_dict = [{'dataset': '../../build_corpus_mamba/datasets/namuwiki', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/KISTI', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_congress2_2', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/traffic', 'weight': 2},
                {'dataset': '../../build_corpus_mamba/datasets/modu_written', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/aihub_expertise', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/korean_petitions', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_congress2_1', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_congress', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/112_end', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_news', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_conversation', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/kowiki', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/aihub_web', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/kcbert', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/manual_pdf', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/laws', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/modu_conversation2', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/manual_hwp', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/manual', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/mc4', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/112_rpt', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/lbox', 'weight': 2},
                {'dataset': '../../build_corpus_mamba/datasets/aihub_books', 'weight': 1},
                {'dataset': '../../build_corpus_mamba/datasets/police_journal/', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/psi_research/', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/knpu_text_pdf/', 'weight': 3},
                {'dataset': '../../build_corpus_mamba/datasets/laws_summary/', 'weight': 3}
               ]

In [7]:
len(dataset_dict)

28

### 학습용 데이터셋 준비

In [8]:
ds_train_list = []
ds_test_list = []
for item in tqdm(dataset_dict):
    ds = load_from_disk(item['dataset'])
    ds = ds.filter(lambda x: len(x["document"].strip()) > 10)
    ds_dict = ds.shuffle().train_test_split(test_size=0.1)
    for _ in range(0, item['weight']):
        ds_train_list.append(ds_dict['train'])
        ds_test_list.append(ds_dict['test'])

100%|██████████| 28/28 [00:35<00:00,  1.27s/it]


In [9]:
len(ds_train_list)

46

In [10]:
len(ds_test_list)

46

In [11]:
ds_train = concatenate_datasets(ds_train_list).shuffle()

In [12]:
ds_test = concatenate_datasets(ds_test_list).shuffle()

In [13]:
ds_train

Dataset({
    features: ['document'],
    num_rows: 82711427
})

In [14]:
ds_test

Dataset({
    features: ['document'],
    num_rows: 9190181
})

In [15]:
ds_test['document'][0]

'화명소방서 건너편 / 화명인터리어 내 도박 문을 잠그고 현관에는 사람들이 tv를 보고 있고 방안에서 도박한다 / 지난번 신고를 하니 문이 잠겼다고 경찰이 그냥갔다\n'

In [19]:
tokenized_ds_train = ds_train.map(tokenize3, batched=True, num_proc=4, remove_columns=ds_train.column_names)

Map (num_proc=4): 100%|██████████| 82711427/82711427 [2:18:28<00:00, 9954.48 examples/s]   


In [20]:
tokenized_ds_train

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9597793
})

In [21]:
tokenized_ds_test = ds_test.map(tokenize3, batched=True, num_proc=4, remove_columns=ds_test.column_names)

Map (num_proc=4): 100%|██████████| 9190181/9190181 [14:05<00:00, 10873.77 examples/s]


In [22]:
tokenized_ds_test

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1065969
})

In [23]:
ds_dict = DatasetDict({'train': tokenized_ds_train, 'test':tokenized_ds_test})

In [24]:
ds_dict.save_to_disk('./data/tokenized_dataset_130g_2k_v4_R5')

Saving the dataset (22/22 shards): 100%|██████████| 1065969/1065969 [00:10<00:00, 103175.79 examples/s]


(1) Version 2 Token 개수

(4673481+519124)*4096 = 21268910080 (21.3B)


(2) Version 3 Token 개수

(4772510+531549)*4096 = 21725425664 (21.7B)

(9594367+1064796)*2028 = 21616782564 (21.6B)

(3) Version 4 Token 개수 - mamba tokenizer 활용

(12303046+1365136)*2028 = 27719073096 (27.7B)


(4) 130g_2k_V4_R5 (new_line, V1, 51200 )

(9597793+1065969)*2028 = 21626109336 (21.6B)

In [25]:
(9597793+1065969)*2028

21626109336

In [12]:
with open("../../build_corpus_mamba/extracted/knpu_text_pdf.txt", "r", encoding="utf-8") as f:
    knpu_lines = f.readlines()

In [13]:
knpu_lines

['- 자료를 수집하고, 정리하며, 자료로부터 어떤 통찰을 이끌어 내는 데에 통계가 - 한 마디로 통계학이 무엇이라고 정의하기는 점점 어려워진다. 이는 통계학이 다루고 있 는 자료의 범위와 활용영역이 더 많아지고 넓어지기 때문이다. - 자료에 대한 수집과 정리 및 분석과 이에 대한 해석에 대한 체계화가 필요하며 이러한 체계화를 이루는 것이 통계학이라고 정의할 수 있다. - 불확실한 상황에서 가장 과학적이며 근거 있는 의사결정을 하기 위하여 자료를 수집하고 가공하며 이를 분석하여 해석함으로써 좀 더 합리적인 의사결정을 할 수 있다. - 학문적으로는 실증주의의 영향으로 학술적 주장에 대한 근거제시가 중요하게 됨으로써 자연현상 및 사회현상을 다루는 거의 모든 학문에 통계학 이론이 사용되었다.\n',
 '\n',
 '- 거의 모든 영역에서 통계학 지식이 사용되고 있다. 특히 최근 빅데이터에 대한 관심이 많아지고 있는데, 빅데이터로부터의 의미를 도출하는 데에도 통계학이 사용되고 - 모집단은 관심의 대상이 되는 모든 개체의 집합이다. 예) 올 해 수능점수의 평균을 알고 싶을 경우 수능시험을 치룬 모든 학생들이 모집단이 - 표본은 모집단에 대한 접근이 곤란할 경우 관심 대상이 되는 집단에서 일부를 추출한 개 예) 수능시험을 치룬 모든 학생들을 조사할 수 없으므로 일부를 뽑아서 수능점수를 조사 - 모수는 모집단의 특성을 수치로 나타낸 것을 말한다. 평균를 모수로 제시한다. 그 외에도 관찰한 수는 N으로 표시한다. - 통계량은 표본집단의 특성을 수치로 나타낸 것을 말한다. 평균를 통계량으로 제시한다. 그 외에 관찰한 수는 으로 표시한다.\n',
 '\n',
 '어떤 관계나 범위 안에서 여러 가지 값으로 변할 수 있는 수를 변수라고 한다. 예컨대 학생 들의 신체발달에 따른 연구를 하기 위해 학생들의 키를 측정한다고 할 경우 시간의 경과에 따라서 학생들의 키가 변하는 경우가 이에 해당된다. 변하지 않는 수는 상수라고 변수가 가지고 있는 정보의 수준에 따라 변수를 구분할 수 있다.

In [14]:
ds_knpu = Dataset.from_dict({'document': knpu_lines})

In [15]:
with open("../../build_corpus_mamba/extracted/laws_대법원.txt", "r", encoding="utf-8") as f:
    laws_lines = f.readlines()

In [16]:
ds_laws = Dataset.from_dict({'document': laws_lines})

In [ ]:
ds_knpu.save_to_disk('../../build_corpus_mamba/dataset/knpu_text_pdf')

In [22]:
ds_laws.save_to_disk('../../build_corpus_mamba/datasets/laws_summary')

Saving the dataset (0/1 shards):   0%|          | 0/19097 [00:00<?, ? examples/s]

In [14]:
dataset_dict = [{'dataset': ds_knpu, 'weight': 1},
                {'dataset': ds_laws, 'weight': 1}]

In [15]:
ds_train_list = []
ds_test_list = []
for item in tqdm(dataset_dict):
    ds = item['dataset']
    ds = ds.filter(lambda x: len(x["document"].strip()) > 10)
    ds_dict = ds.shuffle().train_test_split(test_size=0.1)
    for _ in range(0, item['weight']):
        ds_train_list.append(ds_dict['train'])
        ds_test_list.append(ds_dict['test'])

  0%|                                                                                                            | 0/2 [00:00<?, ?it/s]

Filter:   0%|          | 0/15879 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19097 [00:00<?, ? examples/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.32it/s]


In [16]:
len(ds_train_list)

2

In [17]:
len(ds_test_list)

2

In [18]:
ds_train = concatenate_datasets(ds_train_list).shuffle()

In [19]:
ds_test = concatenate_datasets(ds_test_list).shuffle()

In [20]:
ds_train

Dataset({
    features: ['document'],
    num_rows: 15724
})

In [21]:
ds_test

Dataset({
    features: ['document'],
    num_rows: 1748
})

In [22]:
tokenized_ds_train = ds_train.map(tokenize2, batched=True, remove_columns=ds_train.column_names)

Map:   0%|          | 0/15724 [00:00<?, ? examples/s]

In [23]:
tokenized_ds_train

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1344
})

In [24]:
tokenized_ds_test = ds_test.map(tokenize2, batched=True, remove_columns=ds_test.column_names)

Map:   0%|          | 0/1748 [00:00<?, ? examples/s]

In [25]:
tokenized_ds_test

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 147
})

In [26]:
ds_dict = DatasetDict({'train': tokenized_ds_train, 'test':tokenized_ds_test})

In [27]:
ds_dict.save_to_disk('./data/tokenized_dataset_130g_2k_v3_addon')

Saving the dataset (0/1 shards):   0%|          | 0/1344 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/147 [00:00<?, ? examples/s]

(1344+147)*2028 = 3023748 (0.003 B)

In [28]:
(1344+147)*2028

3023748